In [ ]:
from numba import jit
import numpy as np
import time

x = np.arange(10000).reshape(100, 100)

@jit(nopython=True, fastmath = True, parallel = True)
def go_fast(a): # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

In [ ]:
@jit(nopython = True, parallel = True, fastmath = True) #, fastmath = True, parallel = True)
def test_fast():
    
    rows    = 10000
    columns = 10000
    
    result = np.arange(rows*columns).reshape(rows, columns)
    for i in range(rows):
        for j in range(columns):

            result[i][j] = i*j

    return result


In [ ]:
test_fast()

In [ ]:
test_fast()

In [ ]:

from numba import jit, cuda
import numpy as np
import plotly.express as px

@jit(nopython = True)
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            return i

    return 255

@jit(nopython = True)
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x] = color

    return image

image = np.zeros((2000 * 2, 2000 * 2), dtype=np.uint8)
img = create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20)
px.imshow(img, color_continuous_scale = "emrld")
